**Block 1: Setup (Run Once)**


In [1]:
# BLOCK 1: SETUP
import os
from google.colab import drive

# 1. Mount Drive (So we can see the .pth file)
drive.mount('/content/drive')

# 2. Install Website Tools
print("Installing Streamlit and Ngrok...")
!pip install -q streamlit pyngrok

!pip install -q reportlab

# 3. Verify Model Path
model_path = "/content/drive/MyDrive/Retinopathy_Project/best_model_pro.pth"
if os.path.exists(model_path):
    print(f"✅ SUCCESS: Model found at {model_path}")
else:
    print(f"❌ ERROR: Model NOT found at {model_path}")
    print("Please check your Drive folders.")

Mounted at /content/drive
Installing Streamlit and Ngrok...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.3 MB/s eta 0:00:00
✅ SUCCESS: Model found at /content/drive/MyDrive/Retinopathy_Project/best_model_pro.pth


**Block 2: The Master App Code (Run This to Build the Site)**

This fixes the "Duplicate Element" error by overwriting the file fresh every time. You can edit the text inside the quotes "" to change how your website looks.

In [2]:
# BLOCK 2: OCUPATH DIAGNOSTICS
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import cv2
import torch.nn.functional as F
import datetime
import io
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

# A. PAGE CONFIGURATION
st.set_page_config(
    page_title="DiaRet-Scan AI",
    page_icon=None,
    layout="wide"
)

# B. CUSTOM CSS (SMART FONT HANDLING)
st.markdown("""
    <style>
        /* 1. Import Google Fonts */
        @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap');

        /* 2. Apply Font to TEXT elements only (Avoids breaking icons) */
        html, body, p, label, h1, h2, h3, h4, h5, h6, li, a, input, textarea {
            font-family: 'Roboto', 'Segoe UI', 'Helvetica Neue', sans-serif !important;
        }

        /* 3. Button Font (But exclude the close/arrow buttons) */
        button:not([kind="header"]) {
            font-family: 'Roboto', 'Segoe UI', 'Helvetica Neue', sans-serif !important;
        }

        /* 4. Background: Deep Dark */
        .stApp {
            background-color: #131314;
        }

        /* 5. Center Main Layout */
        .main .block-container {
            max_width: 1200px;
            padding-top: 4rem;
            text-align: center;
        }

        /* 6. Upload Box Styling (Gemini Search Bar Style) */
        [data-testid='stFileUploader'] {
            background-color: #1E1F20;
            border: 1px solid #444746;
            border-radius: 24px;
            padding: 30px;
            text-align: center;
        }

        /* 7. History Buttons Styling */
        div.stButton > button {
            width: 100%;
            text-align: left;
            background-color: transparent;
            border: none;
            color: #E3E3E3;
            border-bottom: 1px solid #444;
            border-radius: 0;
        }
        div.stButton > button:hover {
            background-color: #2D2E30;
            color: #fff;
        }

        /* 8. Fix Center Alignment for Metrics */
        div[data-testid="stMetricValue"] {
            justify-content: center;
        }
    </style>
""", unsafe_allow_html=True)

# C. MEDICAL KNOWLEDGE BASE
MEDICAL_INFO = {
    "No DR": {
        "title": "No Diabetic Retinopathy Detected",
        "description": "The retinal fundus examination reveals a structurally normal retina with no visible vascular pathology. The optic disc demonstrates sharp, distinct margins, and the macula appears clear without edema. There is an absence of microaneurysms, hemorrhages, or lipid exudates. The vascular arcades maintain a healthy caliber with no signs of tortuosity or beading. However, diabetes can induce cellular-level neurovascular changes before they are visible. Therefore, annual dilated eye exams are strongly recommended to monitor for any subclinical progression.",
        "link": "https://www.aao.org/eye-health/diseases/what-is-diabetic-retinopathy"
    },
    "Mild": {
        "title": "Mild Non-Proliferative Diabetic Retinopathy",
        "description": "This is the earliest clinical stage of diabetic retinopathy. It is pathologically characterized by the presence of at least one microaneurysm—tiny, balloon-like swellings in the retinal capillary walls. These occur because high blood sugar levels damage the pericytes (supporting cells) of the blood vessels, causing them to weaken. While vision is typically unaffected at this stage and no treatment is usually required, this serves as a critical warning sign that microvascular damage has begun. Strict glycemic and blood pressure control can halt progression.",
        "link": "https://www.aao.org/eye-health/diseases/what-is-diabetic-retinopathy"
    },
    "Moderate": {
        "title": "Moderate Non-Proliferative Diabetic Retinopathy",
        "description": "The disease has progressed to an intermediate stage where retinal blood vessel damage is more extensive. Clinical findings typically include multiple microaneurysms, dot-and-blot hemorrhages (deeper retinal bleeding), and hard exudates (lipid deposits from leaking vessels). You may also see Cotton Wool Spots, which are fluffy white patches indicating localized nerve fiber infarction due to ischemia (lack of oxygen). The retina is under stress and may begin signaling for new blood vessel growth. Close monitoring is essential to prevent vision loss.",
        "link": "https://www.aao.org/eye-health/diseases/what-is-diabetic-retinopathy"
    },
    "Severe": {
        "title": "Severe Non-Proliferative Diabetic Retinopathy",
        "description": "This is a critical pre-proliferative stage characterized by significant retinal ischemia. The diagnosis is often based on the '4-2-1 Rule': severe hemorrhages in all four quadrants, significant venous beading (irregular, sausage-like vessel shape) in two quadrants, or moderate Intraretinal Microvascular Abnormalities (IRMA) in one quadrant. At this stage, the oxygen-starved retina is releasing high levels of VEGF (Vascular Endothelial Growth Factor), signaling the body to grow new vessels. Immediate referral to a retina specialist is required as the risk of progression to proliferative disease is very high.",
        "link": "https://www.asrs.org/patients/retinal-diseases/3/diabetic-retinopathy"
    },
    "Proliferative": {
        "title": "Proliferative Diabetic Retinopathy (PDR)",
        "description": "This is the most advanced and sight-threatening stage. It is defined by Neovascularization—the growth of abnormal, fragile new blood vessels on the retina or optic disc. Because these vessels are weak, they frequently rupture, causing Vitreous Hemorrhage (bleeding into the eye's gel) which can cause sudden, severe vision loss. Furthermore, scar tissue accompanying these vessels can contract and cause a Tractional Retinal Detachment. Immediate medical intervention (Laser Photocoagulation or Anti-VEGF injections) is critical to preserve vision.",
        "link": "https://www.nei.nih.gov/learn-about-eye-health/eye-conditions-and-diseases/diabetic-retinopathy"
    }
}

# D. HELPER FUNCTIONS
def generate_pdf(scan_data, medical_info):
    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter

    # Header
    c.setFont("Helvetica-Bold", 22)
    c.drawString(50, height - 50, "DiaRet-Scan AI: Clinical Report")

    c.setFont("Helvetica", 12)
    c.drawString(50, height - 80, f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}")
    c.drawString(50, height - 100, f"Scan ID: {scan_data['id']}")
    c.line(50, height - 110, width - 50, height - 110)

    # Diagnosis
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, height - 150, "Diagnosis Assessment:")

    if medical_info['title'] == "No Diabetic Retinopathy Detected":
         c.setFillColorRGB(0, 0.5, 0)
    else:
         c.setFillColorRGB(0.8, 0, 0)

    c.setFont("Helvetica-Bold", 14)
    c.drawString(50, height - 175, f"{medical_info['title']}")

    c.setFillColorRGB(0, 0, 0)
    c.setFont("Helvetica", 12)
    c.drawString(50, height - 200, f"AI Certainity Score: {scan_data['score']:.2f}%")

    # Images
    try:
        # Original
        img_byte_arr = io.BytesIO()
        scan_data['pil_img_orig'].save(img_byte_arr, format='JPEG')
        img_byte_arr.seek(0)
        c.drawImage(ImageReader(img_byte_arr), 50, height - 420, width=200, height=200, preserveAspectRatio=True)
        c.drawString(100, height - 435, "Raw Fundus Scan")

        # Processed
        display_pil = Image.fromarray(scan_data['display_img'])
        img_byte_arr_2 = io.BytesIO()
        display_pil.save(img_byte_arr_2, format='JPEG')
        img_byte_arr_2.seek(0)
        c.drawImage(ImageReader(img_byte_arr_2), 300, height - 420, width=200, height=200, preserveAspectRatio=True)
        c.drawString(350, height - 435, "Enhanced Analysis")
    except:
        c.drawString(50, height - 350, "[Error rendering images]")

    # Description Text
    c.setFont("Helvetica", 11)
    text_obj = c.beginText(50, height - 480)

    # Wrap text
    desc = medical_info['description']
    words = desc.split()
    line = ""
    for word in words:
        if c.stringWidth(line + " " + word) < 500:
            line += " " + word
        else:
            text_obj.textLine(line)
            line = word
    text_obj.textLine(line)
    c.drawText(text_obj)

    # Reference Link
    c.setFont("Helvetica-Oblique", 10)
    c.setFillColorRGB(0, 0, 1) # Blue
    c.drawString(50, height - 600, f"Reference: {medical_info['link']}")

    c.save()
    buffer.seek(0)
    return buffer

# E. STATE MANAGEMENT
if 'history' not in st.session_state:
    st.session_state.history = []
if 'current_view' not in st.session_state:
    st.session_state.current_view = None

def save_scan(uploaded_file, pil_img_orig, display_img, diagnosis, score):
    scan_data = {
        "id": datetime.datetime.now().strftime("%H:%M:%S"),
        "name": uploaded_file.name,
        "pil_img_orig": pil_img_orig,
        "display_img": display_img,
        "diagnosis": diagnosis,
        "score": score
    }
    st.session_state.history.insert(0, scan_data)
    if len(st.session_state.history) > 10:
        st.session_state.history.pop()
    st.session_state.current_view = scan_data

# F. BACKEND LOGIC
@st.cache_resource
def load_model():
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = models.resnet50(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 5)
        load_path = "/content/drive/MyDrive/Retinopathy_Project/best_model_pro.pth"
        if torch.cuda.is_available():
            state_dict = torch.load(load_path)
        else:
            state_dict = torch.load(load_path, map_location=torch.device('cpu'))
        model.load_state_dict(state_dict)
        model.to(device)
        model.eval()
        return model, device
    except Exception as e:
        return None, str(e)

def process_image_ben_graham(image_file):
    # 1. Read Bytes & Decode
    file_bytes = np.asarray(bytearray(image_file.read()), dtype=np.uint8)
    img_bgr = cv2.imdecode(file_bytes, 1)

    # 2. Convert to RGB (Original)
    img_rgb_orig = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # 3. Processing (Work on Copy)
    img = img_rgb_orig.copy()

    tol=7
    if img.ndim == 2:
        mask = img > tol
        img = img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        check_shape = img[:,:,0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if (check_shape != 0):
            img1=img[:,:,0][np.ix_(mask.any(1), mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1), mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)

    img = cv2.resize(img, (224, 224))
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0,0), 10), -4, 128)

    return Image.fromarray(img_rgb_orig), img

# G. MAIN UI

# 1. SIDEBAR HISTORY
st.sidebar.markdown("### Recent Scans")
for i, scan in enumerate(st.session_state.history):
    label = f"{scan['diagnosis']} | {scan['id']}"
    if st.sidebar.button(label, key=f"hist_{i}"):
        st.session_state.current_view = scan
st.sidebar.markdown("---")

# 2. MAIN LAYOUT
left_co, cent_co, last_co = st.columns([1, 2, 1])
with cent_co:
    st.markdown(
        """
        <h1 style='text-align: center; background: -webkit-linear-gradient(45deg, #4285F4, #9B72CB); -webkit-background-clip: text; -webkit-text-fill-color: transparent;'>
            DiaRet-Scan AI
        </h1>
        """,
        unsafe_allow_html=True
    )
    st.markdown("<p style='text-align: center; color: #aaa; margin-bottom: 20px;'>Advanced Diabetic Retinopathy Screening System</p>", unsafe_allow_html=True)
    uploaded_file = st.file_uploader("", type=["jpg", "png", "jpeg"], label_visibility="collapsed")

# 3. LOGIC
if uploaded_file is not None:
    if st.session_state.current_view is None or st.session_state.current_view['name'] != uploaded_file.name:
        pil_img_orig, display_img = process_image_ben_graham(uploaded_file)

        model, device = load_model()
        if model:
            processed_pil = Image.fromarray(display_img)
            transform = transforms.Compose([transforms.ToTensor()])
            img_tensor = transform(processed_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                outputs = model(img_tensor)
                probs = F.softmax(outputs, dim=1)
                confidence, predicted_class = torch.max(probs, 1)

            class_names = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative']
            diag = class_names[predicted_class.item()]
            score = confidence.item() * 100

            save_scan(uploaded_file, pil_img_orig, display_img, diag, score)

# 4. RESULTS VIEW
if st.session_state.current_view is not None:
    data = st.session_state.current_view
    info = MEDICAL_INFO[data['diagnosis']]

    st.divider()

    # Images
    c1, img1, img2, c2 = st.columns([0.5, 3, 3, 0.5])

    with img1:
        st.image(data['pil_img_orig'], width=350)
        st.markdown("<p style='text-align: center; color: #888;'>Original Scan</p>", unsafe_allow_html=True)

    with img2:
        st.image(data['display_img'], width=350)
        st.markdown("<p style='text-align: center; color: #888;'>AI Analysis View</p>", unsafe_allow_html=True)

    st.divider()

    # Text Results (Side by Side)
    r1, r2 = st.columns([2, 1])

    with r1:
        t1, t2 = st.columns([0.6, 1])
        with t1:
            st.markdown("### Clinical Assessment:")
        with t2:
            if data['diagnosis'] == "No DR": color = "#4caf50"
            elif data['diagnosis'] in ["Mild", "Moderate"]: color = "#ff9800"
            else: color = "#f44336"
            st.markdown(f"<h3 style='color:{color}; margin-top:0;'>{data['diagnosis']}</h3>", unsafe_allow_html=True)

        st.write(info['description'])
        st.markdown(f"[View Clinical Guidelines]({info['link']})")

    with r2:
        st.subheader("Model Probability")
        st.metric(label="Eye Health Prediction", value=f"{data['score']:.2f}%")
        st.progress(data['score'] / 100)

        pdf_file = generate_pdf(data, info)
        st.download_button(
            label="Download PDF Report",
            data=pdf_file,
            file_name=f"Report_{data['id']}.pdf",
            mime="application/pdf"
        )

Writing app.py


**Block 3: Launch (Run Last)**


In [4]:
# BLOCK 3: LAUNCH
from pyngrok import ngrok

# 1. Authenticate
NGROK_AUTH_TOKEN = "38Ki6FTWcsmFhVPhHlro0B79alb_7o3oCUVqrGADYH18iQWTw"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 2. Kill old sessions
ngrok.kill()

# 3. Open Tunnel
public_url = ngrok.connect(8501).public_url
print(f"🚀 YOUR APP IS LIVE HERE: {public_url}")

# 4. Run App
!streamlit run app.py &>/dev/null&

🚀 YOUR APP IS LIVE HERE: https://gavin-febrile-culpably.ngrok-free.dev
